In [1]:
# Web에 포함시킬 모델을 구성 
import pandas as pd 

In [2]:
df1 = pd.read_csv('01_Data.csv')

In [19]:
df1

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,...,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank,Target
0,1,66758234,렌탈,일반계약,영업방판,2019-05-06,60,CMS,DES-1,96900,...,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고,0
1,2,66755948,렌탈,교체계약,영업방판,2020-02-20,60,카드이체,DES-1,102900,...,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드,0
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2019-02-28,60,CMS,DES-1,96900,...,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행,0
3,4,66423450,멤버십,멤버십3유형,재계약,2019-05-13,12,CMS,DES-1,66900,...,39.0,경기도,경기도,계약확정,0,없음,남자,5.0,농협회원조합,0
4,5,66423204,멤버십,멤버십3유형,재계약,2019-05-10,12,CMS,DES-1,66900,...,60.0,경기도,경기도,기간만료,12,있음,남자,8.0,농협회원조합,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51296,51298,66579515,렌탈,프로모션계약,대형마트A,2019-03-01,60,CMS,DES-3A,96900,...,47.0,경기도,경기도,계약확정,0,없음,남자,NaN,기업은행,0
51297,51299,66799558,렌탈,일반계약,대형마트A,2019-04-01,60,CMS,DES-1,96900,...,42.0,경기도,경기도,계약확정,0,없음,여자,8.0,새마을금고,0
51298,51300,66799197,렌탈,프로모션계약,영업방판,2019-04-01,39,카드이체,ERA,120900,...,65.0,서울특별시,서울특별시,계약확정,0,없음,여자,1.0,롯데카드,0
51299,51301,66792778,렌탈,일반계약,홈쇼핑/방송,2020-02-06,60,카드이체,DES-1,96900,...,54.0,서울특별시,서울특별시,계약확정,0,없음,여자,2.0,롯데카드,0


In [5]:
df1['Target']= df1['State'].replace({'계약확정':0,
                                     '기간만료':0,
                      '해약확정':1,'해약진행중':1})

In [8]:
X = df1[['Term', 'Product_Type', 'Amount_Month', 
         'Age','Gender','Credit_Rank']]
Y = df1['Target']

- RandomForestClassifier 
- MinMaxScaling / OneHotEncoder 
- SimpleImputer 
- SMOTE
- CV = 3 / max_depth 5~10 / min_samples_split 5~10 
- model_web.sav

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

In [11]:
X_train, X_test, Y_train, Y_test =train_test_split(X,Y,
                                    random_state=1234)

In [16]:
numeric_pipe = make_pipeline(SimpleImputer(),
                             MinMaxScaler())
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'),
                             OneHotEncoder())
numeric_list = X.describe().columns.tolist()
category_list = X.describe(include='object').columns.tolist()
preprocess_pipe = make_column_transformer( (numeric_pipe, numeric_list),
                                         (category_pipe, category_list))
pipe_model = make_pipeline(preprocess_pipe, SMOTE(),
                          RandomForestClassifier())

In [17]:
hyper_list = {'randomforestclassifier__max_depth':range(5,11),
             'randomforestclassifier__min_samples_split':range(5,11)}
grid_model = GridSearchCV(pipe_model, param_grid=hyper_list, cv=5,
                         scoring='f1', n_jobs=-1)
grid_model.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('minmaxscaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Term',
                                                                          'Amount_Month',
                                                                          'Age',
                                                                          'Credit_Rank']),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder())]),
                                                                         ['Product_Type',
                                                                          'Gender'])])),
                                       ('smote', SMOTE()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'randomforestclassifier__max_depth': range(5, 11),
                         'randomforestclassifier__min_samples_split': range(5, 11)},
             scoring='f1')

In [18]:
best_model = grid_model.best_estimator_
pickle.dump(best_model, open('model_web.sav','wb'))

In [175]:
best_model.predict_proba(X_train)[5][1]

0.4268934751850747

In [176]:
best_model.predict_proba(X_train)

array([[0.75392075, 0.24607925],
       [0.56983629, 0.43016371],
       [0.86731002, 0.13268998],
       ...,
       [0.70209658, 0.29790342],
       [0.53357836, 0.46642164],
       [0.5898339 , 0.4101661 ]])

# 팀 설정 

In [177]:
import random

In [178]:
def Team_Selection():
    random.shuffle(a_list)
    random.shuffle(b_list)
    random.shuffle(c_list)
    random.shuffle(d_list)
    random.shuffle(e_list)
    df_Team = pd.DataFrame([a_list, b_list, c_list,d_list],
                 columns=['A팀','B팀','C팀','D팀'])
    return df_Team

In [183]:
Team_Selection()

,A팀,B팀,C팀,D팀
0,이준엽,김영우,남정윤,이용석
1,박상범,정연재,구본하,안주강
2,박태윤,양승호,변지은,정예은
3,박은영,이다련,김현영,전준우
4,,김예슬,,
